# Import ELK Snapshots onto a Fabric slice

### Prerequisite
- A running fabric slice (For example: [Create single node backup Slice](./create_single_node_backup.ipynb))
- An ELK snapshot tar file to import

#### General Imports

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()                
except Exception as e:
    print(f"Exception: {e}")

#### Get your slice information
*Set slice_name and node_name below before running*

In [ ]:
slice_name='backup-testing'
node_name='data-backup'


slice = fablib.get_slice(name=slice_name)
slice.show();
node = slice.get_node(name=node_name)
node.show();

#### Import ELK service tool

In [ ]:
from mflib.data_transfer import ElkImporter
elk_tools = ElkImporter(slice_name=slice_name, node_name=node_name)

# Instructions to start ELK and import your snapshot data

#### Ensure Docker and Docker-Compose are installed on the node

In [ ]:
elk_tools.install_docker()

#### Set up NAT64 Server
We need to do this step so that we can clone the Github repo with our docker app.

In [ ]:
elk_tools.setup_nat64(node_name)

#### Clone Docker App Repository
Clones https://github.com/fabric-testbed/mf-data-import-containers to your docker container.
This repository contains a docker-compose app which will allow you to automatically set up Elastic Search and Kibana on your node.

In [ ]:
elk_tools.clone_repository()

---
### Upload snapshot tar file to your node

#### Option 1: Upload to node from local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# The ssh config file location on your local device
ssh_config = "~/.ssh/fabric/ssh_config"

# The fabric slice private key file on your local device
private_key = "~/.ssh/fabric/slice_key"

# The Nodes username
username = node.get_username()

# The Nodes IP Address
ip = node.get_management_ip()

# Destination to store snapshot file on your local device
local_file_location = "/Your/local/path/here"

# Snapshot file name
snapshot_file_name = "snapshot_test.tar"

# Create and print command
scp_command = f"scp -F {ssh_config} -i {private_key} {local_file_location}{snapshot_file_name} {username}@\[{ip}]:/tmp/{snapshot_file_name}"
print(scp_command)

#### Once the file is on the meas_node, copy it to the right directory

In [ ]:
node.execute(f'sudo mkdir /home/ubuntu/data-import-containers/elk/imported_data')
node.execute(f'sudo tar -xvf /tmp/{snapshot_file_name} -C /home/ubuntu/data-import-containers/elk/imported_data')

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

### Option 2: Upload from Jupyter Hub server

##### Use this method if you have your tar file in the snapshots directory on this JH server

In [ ]:
# Enter your elk snapshot tar file name (must be in data_transfer_service/snapshots directory)
snapshot_tar_name = "snapshot_test.tar"

# 1. Uploads TAR file to your node
# 2. Untars data and places inside ELK's shared docker volume
elk_tools.import_snapshot(snapshot_tar_name)

In [ ]:
# NEED TO ADD THIS - sudo tar -xvf snapshots/snapshot_test.tar -C imported_data/

---
#### Start the docker compose app

In [ ]:
elk_tools.start_docker()

#### Register the repository

In [ ]:
# Choose a name for the repository your snapshot will be imported into or keep as default
repository_name = "imported_repository"

elk_tools.register_repository(repository_name)

#### View repository after registration

In [ ]:
elk_tools.view_repository("imported_repository")

#### Delete all indices and restore snapshot (replace snapshot name and change index list if you want to customize)

In [ ]:
repository_name = "imported_repository"
snapshot_name = "snapshot_test"

elk_tools.restore_snapshot(repository_name, snapshot_name)

### Your data is now imported on the node.

----
### View the index list

In [ ]:
elk_tools.view_snapshot("imported_repository", "snapshot_test")

In [ ]:
elk_tools.view_indices()

----
### View data from Kibana

#### On your local terminal create a tunnel with:

'ssh -L 10030:localhost:5601 -F ssh_config -i slice_key ubuntu@{node_ip}'

#### Then go to http://localhost:10030
----

### To reuse slice with new data (if you are done then just delete the slice)
#### Take down container and remove data

In [ ]:
elk_tools.stop_docker()
elk_tools.remove_data()

#### Rerun steps starting from *Create Docker container and import your data*